In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 50
CHANNELS = 3

In [4]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [5]:
def get_partitions(ds, train_split=0.8, test_split=0.1, val_split=0.1, shuffle=True, shuffle_size=10000):
    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    return train_ds, val_ds, test_ds

In [6]:
train_ds, val_ds, test_ds = get_partitions(dataset)

In [7]:
train_ds = train_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2)
])

In [9]:
vgg19_model = VGG19(include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS))

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 508s 6us/step


In [10]:
for layer in vgg19_model.layers:
    layer.trainable = False


In [11]:
model_vgg19 = models.Sequential([
    data_augmentation,
    vgg19_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])


In [12]:
model_vgg19.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
history_vgg19 = model_vgg19.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 589s 11s/step - accuracy: 0.4417 - loss: 4.9835 - val_accuracy: 0.7969 - val_loss: 0.9834
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1480s 28s/step - accuracy: 0.6659 - loss: 2.0450 - val_accuracy: 0.8542 - val_loss: 0.7446
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 39s/step - accuracy: 0.7357 - loss: 1.4064 

KeyboardInterrupt: 

In [ ]:
#validation of Model
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns

# Predict the labels for the test dataset
test_preds = model_vgg19.predict(test_ds)
test_preds_labels = np.argmax(test_preds, axis=1)

# True labels for the test dataset
test_labels = np.concatenate([y for x, y in test_ds], axis=0)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_preds_labels)

# Compute precision and recall
precision = precision_score(test_labels, test_preds_labels, average='weighted')
recall = recall_score(test_labels, test_preds_labels, average='weighted')

print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Class 0', 'Class 1', 'Class 2'],
            yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
